In [1]:
# My character's carrying capacity
my_capacity = 260  # in weight
my_slots = 35      # in items, if they're not stackable

In [2]:
import pandas as pd
from difflib import get_close_matches
from tqdm import tqdm
tqdm.pandas(desc = 'Fuzzy matching')

# Define the fuzzy merging method for dataframes
def fuzzymerge(self, right, on = None, left_on = None, right_on = None, 
               how = 'left', cutoff = 0):        
    '''Fuzzy merging. Same as standard pandas merge, but the `on` columns
    do not have to match exactly. Parameter `cutoff` controls this, e.g.: 
         * If cutoff = 0 (default), a match is always found.
         * If cutoff = 1, only a perfect match is acceptable.'''

    # Preliminary handling of `on` parameters
    if on is not None:
        left_on, right_on = on, on
    elif left_on is None or right_on is None:
        raise ValueError('Either `on`, or both `left_on` '
                         'and `right_on`, must be specified.')
    if type(left_on) != str or type(right_on) != str:
        raise TypeError('Must fuzzymerge on a single '
                        'column name in each frame.')

    # Fuzzy matching function
    right_values = right[right_on].unique()
    def fm(s):
        ret = get_close_matches(s, right_values, 1, cutoff)
        try: return ret[0]
        except: return None

    # Build intermediate dataframe for matching
    matches = self[[left_on]].drop_duplicates()
    matches['__matchcol__'] = matches[left_on].progress_apply(fm)        
    matches.dropna(subset = ['__matchcol__'], inplace = True)

    # Return results
    return self.merge(matches, on = left_on, how = how).merge(
        right, how = how, 
        left_on = '__matchcol__', 
        right_on = right_on
    ).drop('__matchcol__', 1)        

# Add fuzzy merging capability to all dataframes
pd.DataFrame.fuzzymerge = fuzzymerge

In [3]:
# Get the entire trading details of all bots & NPC's (this will take a moment)
df = pd.read_html(
    'http://greypal.el-fd.org/cgi-bin/querybot?action=Both&item=.%2B',
    header = 0
)[0]

# We don't care about owners or hosters of the bots
df = df.drop(['Hoster', 'Owner'], 1)

In [4]:
# Clean up the amount available to be bought or sold
max_amount = df[df.Amount != '*'].Amount.astype(int).max()
df['Amount'] = df.Amount.apply(lambda s: max_amount if s == '*' else int(s))

# Strip the units ("gc" for "gold coins") from the prices
df['Price'] = df.Price.str[:-2].astype(float)

In [5]:
# Create a dataframe of "opportunities" -- where an item 
# being sold by one bot is being bought by another
oppo = df[df.Action == 'Selling'].merge(df[df.Action == 'Buying'], on = 'Item')

# Filter down to cases where the buying bot will 
# pay more than the price of the selling bot
oppo = oppo[oppo.Price_y > oppo.Price_x]

In [6]:
# Load item information scraped from el-wiki: weights and stackability
weights = pd.read_pickle('weights.pkl')

# Attach the item information to each opportunity
# by fuzzy matching the item name to a wiki page title
oppo = oppo.fuzzymerge(weights, left_on = 'Item', right_on = 'WikiPage')

# Take a look at the results of the fuzzy matching
oppo[['Item', 'WikiPage']].drop_duplicates()

Fuzzy matching: 100%|██████████████████████████████████████████████████████████████████| 64/64 [00:15<00:00,  4.06it/s]


,Item,WikiPage
0,Red Rose,Red Rose
1,Sulfur,Sulfur
12,Silver Ore,Silver Ore
28,Bones,Bones
163,Raw Meat,Raw Meat
168,Dung,Dung
176,Thread,Thread
210,Ale,Ale
235,Titanium Ore,Titanium Ore
237,Iron Ore,Iron Ore


In [7]:
# Get the per-unit value of each possible exchange between bots
oppo['UnitValue'] = [p2-p1 for p1,p2 in zip(oppo.Price_x, oppo.Price_y)]

# The total value of each possible exchange (if all units are transferred)
oppo['TotalValue'] = [
    min(a1,a2)*v for a1,a2,v 
    in zip(oppo.Amount_x, oppo.Amount_y, oppo.UnitValue)
]

# The number of units that can be transferred in one trip
oppo['LoadSize'] = [
    min(my_capacity // w if s != 'no' else my_slots, a1, a2)
    for w,s,a1,a2 in zip(
        oppo.Weight, oppo.Stackable, oppo.Amount_x, oppo.Amount_y
    )
]

# The amount that can be gained in one trip
oppo['LoadValue'] = [s*v for s,v in zip(oppo.LoadSize, oppo.UnitValue)]

In [8]:
# View the opportunites with the largest value per load
oppo.sort_values('LoadValue', ascending = False).drop([
    'Action_x', 'Amount_x', 'Price_x', 
    'Action_y', 'Amount_y', 'Price_y', 
    'Weight', 'UnitValue'
], 1).head(20)

,Bot_x,Location_x,Item,Bot_y,Location_y,ItemMatched,Stackable,TotalValue,LoadSize,LoadValue
12359,Locuas,"MM(C1) 84,115",Red Dragon Scale,mufo,"MM(C1) 66,113",Red Dragon Scale,yes,5000.0,20,5000.0
12377,TonyStark,"MM(C1) 68,117",Red Dragon Scale,mufo,"MM(C1) 66,113",Red Dragon Scale,yes,3000.0,20,3000.0
13110,Auriana,"DP(C1) 178,62",Orange,Pru,"DP(C1) 181,116",Orange,yes,2050.0,1,2050.0
12395,Jute,"MM(C1) 70,113",Red Dragon Scale,mufo,"MM(C1) 66,113",Red Dragon Scale,yes,2000.0,20,2000.0
12413,Landerion,"MM(C1) 74,113",Red Dragon Scale,mufo,"MM(C1) 66,113",Red Dragon Scale,yes,1900.0,19,1900.0
9040,NPC-Urania,"Arius(C2) 126,200",Make Rare/Fail Indicator,Adarah,"WS(C1) 466,425",Make Rare/Fail Indicator,yes,1460.0,130,1300.0
11387,NPC-Ringa,"IotF(C2) 84,94",Ring of Anitora,Adarah,"WS(C1) 466,425",Ring of Anitora,yes,1230.0,246,1230.0
11228,busy,"PL(C1) 227,113",Mule Glyph,LumberJack,"Hurquin(C2) 124,80",Mule Glyph,yes,1225.0,35,1225.0
9868,NPC-Clark,"PL(C1) 198,258",Ring of Isla Prima,LumberJack,"Hurquin(C2) 124,80",Ring of Isla Prima,yes,1110.0,111,1110.0
11248,Adarah,"WS(C1) 466,425",Mule Glyph,LumberJack,"Hurquin(C2) 124,80",Mule Glyph,yes,1050.0,35,1050.0


In [9]:
# View the opportunities in more useful form, and cut out all "C2" locations
for _, row in oppo[
    (-oppo.Location_x.str.contains('\(C2\)')) 
    & (-oppo.Location_y.str.contains('\(C2\)'))
].sort_values('LoadValue', ascending = False).head(20).iterrows():
    print(
        '%.0fgc: Buy %d %s from %s (%s) for %.0fgc and sell to %s (%s)' % (
            row.LoadValue, row.LoadSize, 
            row.Item, row.Bot_x, row.Location_x, 
            row.Price_x * row.LoadSize,
            row.Bot_y, row.Location_y
        )
    )

5000gc: Buy 20 Red Dragon Scale from Locuas (MM(C1) 84,115) for 55000gc and sell to mufo (MM(C1) 66,113)
3000gc: Buy 20 Red Dragon Scale from TonyStark (MM(C1) 68,117) for 57000gc and sell to mufo (MM(C1) 66,113)
2050gc: Buy 1 Orange from Auriana (DP(C1) 178,62) for 6450gc and sell to Pru (DP(C1) 181,116)
2000gc: Buy 20 Red Dragon Scale from Jute (MM(C1) 70,113) for 58000gc and sell to mufo (MM(C1) 66,113)
1900gc: Buy 19 Red Dragon Scale from Landerion (MM(C1) 74,113) for 55100gc and sell to mufo (MM(C1) 66,113)
1032gc: Buy 24 Fox Fur from Adarah (WS(C1) 466,425) for 288gc and sell to Raised_by_Bats (TG(C1) 203,148)
500gc: Buy 1 Invasion Token from danius (PL(C1) 211,83) for 13500gc and sell to mufo (MM(C1) 66,113)
368gc: Buy 46 Ring of SRM from Eset (WS(C1) 136,713) for 17940gc and sell to Beaver (PL(C1) 153,96)
350gc: Buy 35 Iron Sword from NPC-Paul (PL(C1) 275,91) for 12250gc and sell to Adarah (WS(C1) 466,425)
325gc: Buy 130 Pking arrows from NPC-Bower (MM(C1) 277,65) for 4550gc an